# To do's
- find out what a good fitness value is
- calculate SSE
- calculate silhouette coefficients
- bring back whales which went out of bounds
- ...

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import norm

In [2]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',header=None)
scaler = MinMaxScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df)
df = df.sample(frac=1)

/Users/kareem/Library/Python/2.7/lib/python/site-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [3]:
df.columns = ["class",
            "alcohol",
            "malic_acid",
            "ash",
            "alcalinity_of_ash",
            "magnesium",
            "total_phenols",
            "flavanoids",
            "nonflavanoid_phenols",
            "proanthocayanins",
            "color_intensity",
            "hue",
            "od280",
            "proline"]
df = df.drop(["class"],axis=1)

In [4]:
def init_whale(spaceDim,numCenters,numPopulation,lowerBound,upperBound):
    whalePop = []
    for i in range(numPopulation):
        tmp = np.random.rand(numCenters*spaceDim)*(upperBound-lowerBound)-(upperBound-lowerBound)/2
        whalePop.append(tmp)
    return whalePop

In [5]:
maxIter = 400
numPopulation = 40
spaceDim = len(df.columns)
P = len(df) #178
numCenters = 3
lowerBound = 0 #must be negativ
upperBound = 1 #must be positive
b = 1.0 #dd0.618
#LSSI = 999999999

In [6]:
whalePop = init_whale(spaceDim,numCenters,numPopulation,lowerBound,upperBound)

In [7]:
t = 0
count = 0
bestWhale = None
currentBestWhale = None
fitness_vec = []

while t<maxIter:        
    
    #STEP 1 ---------------------------------------
    
    fitnessPop = []
    for whale in whalePop:
        
        #Lamarickan extension
        #if t%LSSI == 0:
        #    fitness_avg = [sum(fitness_vec)/len(fitness_vec)]*len(fitness_vec)
        #    difference = fitness_vec-fitness_avg
        #    variance = sum([i**2 for i in difference])/len(fitness_vec)
        #    if variance <= fitness_vec[-1]:
        #        #...

        fitnessWhale = 0
        for p in range(P):
            distances = [norm(df.iloc[p]-whale[0:13]),norm(df.iloc[p]-whale[13:26]),norm(df.iloc[p]-whale[26:39])]
            idxDist = np.argmin(distances)
            fitnessWhale += distances[idxDist]
        fitnessPop.append(fitnessWhale)
        count += 1
    idxFit = np.argmin(fitnessPop)
    bestWhale = whalePop[idxFit]
    fitness_vec.append(fitnessPop[idxFit])
    
    print("Iteration "+str(t)+", Fitness of best whale "+str(fitnessPop[idxFit]))
        
    #STEP 2 ---------------------------------------
    
    for i,whale in enumerate(whalePop,0):
        
        if i != idxFit: #keep best whale
            
            #Update values a,A,C,l,p
            a = 2-(2*t)/maxIter #decreases linearly from 2 to 0
            r = np.ones(numCenters*spaceDim)*np.random.uniform(0,1,1)[0] #a vector filled with the same random number in [0,1]
            A = 2*a*r-a #a vector filled with the same random value in [-a,a]
            C = 2*r
            p = np.random.uniform(0,1,1)[0]
            l = np.random.uniform(-1,1,1)[0]

            if p < 0.5:
                if norm(A) < 1:

                    #Phase of encircling
                    #Note that with increasing iteration exploration will stop
                    #D = np.abs(np.multiply(C,bestWhale)-whale)
                    D = np.multiply(C,bestWhale)-whale
                    whalePop[i] = bestWhale-np.multiply(A,D)

                elif norm(A) >= 1: 

                    #Phase of exploration
                    XRand = np.random.rand(numCenters*spaceDim) #must not necessarily be in [0,1]
                    #D = np.abs(np.multiply(C,bestWhale)-whale)
                    D = np.multiply(C,bestWhale)-whale
                    whalePop[i] = XRand-np.multiply(A,D)

            elif p>= 0.5:

                #Phase of spiral
                D_ = bestWhale-whale
                whalePop[i] = D_*np.exp(b*l)*np.cos(2*np.pi*l)+bestWhale

            #Bring back whale which is out of search space
            for j,coordinate in enumerate(bestWhale,0):
                if coordinate > 1:
                    bestWhale[j] = 0.5
                elif coordinate < 0:
                    bestWhale[j] = 0.5
            
    t += 1

Iteration 0, Fitness of best whale 171.08262164261416
Iteration 1, Fitness of best whale 145.9106758831101
Iteration 2, Fitness of best whale 142.08153488784237
Iteration 3, Fitness of best whale 140.8358723038955
Iteration 4, Fitness of best whale 140.43804823118253
Iteration 5, Fitness of best whale 135.16450011750712
Iteration 6, Fitness of best whale 135.11017260549357
Iteration 7, Fitness of best whale 132.3640380295311
Iteration 8, Fitness of best whale 131.09766025615704
Iteration 9, Fitness of best whale 129.94975925276384
Iteration 10, Fitness of best whale 129.5149661352807
Iteration 11, Fitness of best whale 127.21279721720633
Iteration 12, Fitness of best whale 126.3549353107471
Iteration 13, Fitness of best whale 125.84708700103836
Iteration 14, Fitness of best whale 125.6669892850436
Iteration 15, Fitness of best whale 124.67039316799112
Iteration 16, Fitness of best whale 124.49692212345528
Iteration 17, Fitness of best whale 121.59115089090751
Iteration 18, Fitness of b

Iteration 151, Fitness of best whale 93.07131778689488
Iteration 152, Fitness of best whale 93.06689762553518
Iteration 153, Fitness of best whale 93.06689762553518
Iteration 154, Fitness of best whale 92.91803344887225
Iteration 155, Fitness of best whale 92.84917355926784
Iteration 156, Fitness of best whale 92.84678891214514
Iteration 157, Fitness of best whale 92.84678891214514
Iteration 158, Fitness of best whale 92.80190343233984
Iteration 159, Fitness of best whale 92.80190343233984
Iteration 160, Fitness of best whale 92.80190343233984
Iteration 161, Fitness of best whale 92.80190343233984
Iteration 162, Fitness of best whale 92.743401373482
Iteration 163, Fitness of best whale 92.62324035186741
Iteration 164, Fitness of best whale 92.62324035186741
Iteration 165, Fitness of best whale 92.51723769790861
Iteration 166, Fitness of best whale 92.51715609443137
Iteration 167, Fitness of best whale 92.49069647685624
Iteration 168, Fitness of best whale 92.17999327828957
Iteration 16

Iteration 301, Fitness of best whale 89.23475403095685
Iteration 302, Fitness of best whale 89.23475403095685
Iteration 303, Fitness of best whale 89.2345835612696
Iteration 304, Fitness of best whale 89.2345336141457
Iteration 305, Fitness of best whale 89.2345336141457
Iteration 306, Fitness of best whale 89.23451337524406
Iteration 307, Fitness of best whale 89.22243723803412
Iteration 308, Fitness of best whale 89.19707912443147
Iteration 309, Fitness of best whale 89.19706581490222
Iteration 310, Fitness of best whale 89.19016326002142
Iteration 311, Fitness of best whale 89.19016326002142
Iteration 312, Fitness of best whale 89.19016326002142
Iteration 313, Fitness of best whale 89.18478376419138
Iteration 314, Fitness of best whale 89.17686325549833
Iteration 315, Fitness of best whale 89.17686325549833
Iteration 316, Fitness of best whale 89.17686325549833
Iteration 317, Fitness of best whale 89.17489695978028
Iteration 318, Fitness of best whale 89.17489695978028
Iteration 319

In [8]:
z1,z2,z3 = bestWhale[0:13],bestWhale[13:26],bestWhale[26:39]

In [9]:
cluster = []
for p in range(P):
    distances = [norm(df.iloc[p]-z1),norm(df.iloc[p]-z2),norm(df.iloc[p]-z3)]
    cluster.append(np.argmin(distances)+1)
df["cluster"] = cluster

In [10]:
SSE = 0
df_tmp = df.drop(["cluster"],axis=1)
for p in range(P):
    if df.iloc[p].cluster == 1:
        SSE += norm(df_tmp.iloc[p]-z1)
    if df.iloc[p].cluster == 2:
        SSE += norm(df_tmp.iloc[p]-z2)
    if df.iloc[p].cluster == 3:
        SSE += norm(df_tmp.iloc[p]-z3)
print("SSE: "+str(SSE/len(df))) #best:0.59

SSE: 0.4992986092592671


In [11]:
x = np.sort(df.cluster.values)
count1 = 0
count2 = 0
count3 = 0
for i in range(len(x)):
    if x[i] == 1:
        count1 += 1
    if x[i] == 2:
        count2 += 1
    if x[i] == 3:
        count3 += 1
print(count1,count2,count3) #should be (59,71,48)

(60, 55, 63)
